In [1]:
%pwd

'c:\\Users\\adity\\Desktop\\lungs classification\\research'

In [2]:
import os

In [4]:
os.chdir("../")

In [6]:
%pwd

'c:\\Users\\adity\\Desktop'

In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [9]:
import sys
import os

# Add the 'src' folder to sys.path
sys.path.append('c:/Users/adity/Desktop/Lung-disease-classificatrion/src')

# Now you can import the Classifier module
from Classifier.constants import *
from Classifier.utils.common import read_yaml, create_directories


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\adity\Desktop\lungs classification\myenv\Lib\site-packages\IPython\core\interactiveshell.py", line 3549, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\adity\AppData\Local\Temp\ipykernel_25428\3989677841.py", line 9, in <module>
    from Classifier.utils.common import read_yaml, create_directories
  File "c:\Users/adity/Desktop/Lung-disease-classificatrion/src\Classifier\utils\common.py", line 4, in <module>
    from src.Classifier import logger
ModuleNotFoundError: No module named 'src'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\adity\Desktop\lungs classification\myenv\Lib\site-packages\IPython\core\interactiveshell.py", line 2173, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\adity\Desktop\lungs classification\myenv\Lib\site-packag

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [9]:
import os
import urllib.request as request
import zipfile
from Classifier import logger
from Classifier.utils.common import get_size

In [10]:
pip install gdown

Note: you may need to restart the kernel to use updated packages.


In [11]:
import os
import requests
import gdown
import zipfile
from pathlib import Path
from src.Classifier.utils.common import get_size  # Ensure this function exists
import logging

logger = logging.getLogger(__name__)

class DataIngestion:
    def __init__(self, config):
        self.config = config  # Ensure this is an instance of DataIngestionConfig

    def download_file(self):
        """Downloads a large file from Google Drive using gdown."""
        file_id = "1SXCYe2xggnVpkHGtkS1GH8S_DwHKo3RT"
        url = f"https://drive.google.com/uc?export=download&id={file_id}"
        local_file = self.config.local_data_file

        os.makedirs(os.path.dirname(local_file), exist_ok=True)

        try:
            gdown.download(url, local_file, quiet=False)
            print(f"Download successful: {local_file}")

        except Exception as e:
            print(f"Download failed: {e}")
            raise

    def extract_zip_file(self):
        """
        Extracts the ZIP file into the data directory.
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        # Ensure the ZIP file exists and is not empty
        if not os.path.exists(self.config.local_data_file) or os.path.getsize(self.config.local_data_file) == 0:
            logger.error("ZIP file is missing or empty. Cannot extract.")
            raise ValueError("ZIP file is missing or empty.")

        try:
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
            logger.info(f"Extraction successful! Files extracted to {unzip_path}")

        except zipfile.BadZipFile:
            logger.error("ZIP file is corrupted! Cannot extract.")
            raise ValueError("ZIP file is corrupted.")



In [13]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    
except Exception as e:
    raise e

[2025-03-30 20:44:32,569: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-30 20:44:32,575: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-30 20:44:32,578: INFO: common: created directory at: artifacts]


[2025-03-30 20:44:32,580: INFO: common: created directory at: artifacts/data_ingestion]


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1SXCYe2xggnVpkHGtkS1GH8S_DwHKo3RT
From (redirected): https://drive.google.com/uc?export=download&id=1SXCYe2xggnVpkHGtkS1GH8S_DwHKo3RT&confirm=t&uuid=7ad2c60a-1e07-4565-a7b7-30f77891a2d6
To: c:\Users\adity\Desktop\Lung-disease-classificatrion\artifacts\data_ingestion\data.zip
100%|██████████| 137M/137M [00:28<00:00, 4.87MB/s] 


Download successful: artifacts/data_ingestion/data.zip
[2025-03-30 20:45:08,066: INFO: 324666251: Extraction successful! Files extracted to artifacts/data_ingestion/]
